In [1]:
!pip install tensorflow

^C


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Set working directory
os.chdir('C:/Users/asus/Documents/GitHub/CMSC-197-Miniproject')

# Read JSON file
df = pd.read_json('data/VADER_normalized.json', orient='records')


# Assuming df is your dataframe and it already contains the features and label
X = df.drop(columns=['Label'])  # Features (all columns except 'Label')
y = df['Label']  # Target variable (Label)

# Fill missing values with the mean of each column
X = X.fillna(X.mean())

# Standardize the data (ANNs generally perform better with normalized data)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the target labels if it's a classification problem
y_encoded = to_categorical(y)  # If the labels are 0 or 1, this will create two columns

# Training and test set
train_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=None)

# Build the ANN model
model = Sequential()

# Input layer (equal to the number of features)
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

# Hidden layer
model.add(Dense(32, activation='relu'))

# Output layer (for binary classification, 1 neuron with sigmoid activation)
model.add(Dense(y_train.shape[1], activation='softmax'))  # softmax for multi-class or sigmoid for binary

# Compile the model
model.compile(loss='categorical_crossentropy',  # categorical_crossentropy for multi-class classification
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true = np.argmax(y_test, axis=1)  # Convert true labels to class labels

# Confusion matrix
cm = confusion_matrix(y_test, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

print("\nClassification Report:")
print(classification_report(y_test, predictions))

cv_scores = cross_val_score(nb, X, y, cv=10)

print(f"10-fold Cross Validation Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")